In [24]:
import numpy as np
import pandas as pd

from datetime import datetime
from IPython.display import display
import cv2
import os

import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization
from tensorflow.keras.layers import Activation, Concatenate, GlobalMaxPooling2D, Average
from tensorflow.keras.layers import GlobalAveragePooling2D, Reshape, Permute, multiply
#from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input

from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input

from tensorflow.keras.utils import Sequence
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping, TensorBoard, LambdaCallback
from tensorflow.keras.utils import to_categorical


import imgaug as ia
from imgaug import augmenters as iaa

In [3]:
def create_augmenter(train=True):
    # from https://github.com/aleju/imgaug
    # Sometimes(0.5, ...) applies the given augmenter in 50% of all cases,
    # e.g. Sometimes(0.5, GaussianBlur(0.3)) would blur roughly every second image.
    sometimes = lambda aug: iaa.Sometimes(0.5, aug)

    # Define our sequence of augmentation steps that will be applied to every image
    # All augmenters with per_channel=0.5 will sample one value _per image_
    # in 50% of all cases. In all other cases they will sample new values
    # _per channel_.
    if train:
        seq = iaa.Sequential(
            [
                # apply the following augmenters to most images
                iaa.Fliplr(0.5), # horizontally flip 50% of all images
                iaa.Flipud(0.2), # vertically flip 20% of all images
                # crop images by -5% to 10% of their height/width
                sometimes(iaa.CropAndPad(
                    percent=(-0.05, 0.1),
                    pad_mode=ia.ALL, # random mode from all available modes will be sampled per image.
                    pad_cval=(0, 255) # The constant value to use if the pad mode is constant or the end value to use if the mode is linear_ramp
                )),
                sometimes(iaa.Affine(
                    scale={"x": (0.8, 1.2), "y": (0.8, 1.2)}, # scale images to 80-120% of their size, individually per axis
                    rotate=(-45, 45), # rotate by -45 to +45 degrees
                    shear=(-16, 16), # shear by -16 to +16 degrees
                    cval=(0, 255), # if mode is constant, use a cval between 0 and 255
                    mode=ia.ALL # use any of scikit-image's warping modes (see 2nd image from the top for examples)
                )),
            ],
        )
    else:
        pass
    return seq

In [4]:
# reference: https://www.kaggle.com/mpalermo/keras-pipeline-custom-generator-imgaug
class BaseDataGenerator(Sequence):
    'Generates data for Keras'
    def __init__(self, images_paths, labels, batch_size=64, image_dimensions = (512, 512, 3),
                 shuffle=False, augmenter=None, preprocessor=None,
                 return_label=True, total_classes=None):
        self.labels       = labels              # array of labels
        self.images_paths = images_paths        # array of image paths
        self.dim          = image_dimensions    # image dimensions
        self.batch_size   = batch_size          # batch size
        self.shuffle      = shuffle             # shuffle bool
        self.augmenter      = augmenter           # augmenter
        self.preprocessor = preprocessor
        self.return_label = return_label
        self.total_classes = total_classes
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.images_paths) / self.batch_size))

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.images_paths))
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def gather_batch_item(self, index):
        'Generate one batch of data'
        # selects indices of data for next batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # select data and load images
        images = [cv2.imread(self.images_paths[k]) for k in indexes]

        # preprocess and augment data
        if self.augmenter:
            images = self.augmenter.augment_images(images)

        images= np.array([self.preprocess_image(cv2.resize(img, self.dim[:2])) for img in images])
        
        if self.return_label:
            labels = np.array([self.labels[k] for k in indexes])
            labels = to_categorical(labels, num_classes=self.total_classes)
            return images, labels
        else:
            return images
    
    def __getitem__(self, index):
        return self.gather_batch_item(index)
        
    def preprocess_image(self, images):
        if self.preprocessor is None:
            images = images / 255.
            pass
        else:
            images = self.preprocessor(images)
        return images
    
class MultiOutputDataGenerator(BaseDataGenerator):
    'Generates multiple output data for Keras'
    def __init__(self, images_paths, labels, batch_size=64, image_dimensions = (512, 512, 3),
                 shuffle=False, augmenter=None, preprocessor=None,
                 return_label=True, total_classes=None, output_names=None, tta_augmentors=None):
        # Init parent's parameter
        super().__init__(images_paths,
                labels, batch_size, image_dimensions,
                 shuffle, augmenter, preprocessor,
                 return_label, total_classes)
        
        self.output_names = output_names
        self.tta_augmentors = tta_augmentors
    
    def __getitem__(self, index):
        if self.return_label:
            images, labels = self.gather_batch_item(index)
            output_dict = {}
            # Copy labels to each output name
            for output_name in self.output_names:
                output_dict[output_name] = labels
            if self.tta_augmentors != None:
                images = self.get_tta_images(images)
            return images, output_dict
        else:
            images = self.gather_batch_item(index)
            if self.tta_augmentors != None:
                images = self.get_tta_images(images)
            return images
    def get_tta_images(self, images):
        aug_images = []
        # Original
        aug_images.append(images)
        for augmentor in self.tta_augmentors:
            aug_images.append(augmentor.augment_images(images))
        images = aug_images
        return images

In [5]:
# pretrain model input size
image_shape = (224, 224, 3)

# 133
total_classes = len(os.listdir('dogImages/train')) 
batch_size = 20

In [6]:
def create_path_labels(path):
    all_classes = os.listdir(path)
    label_dict = {}
    img_paths, label_list = [], []
    for label_name in all_classes:
        label_num, dog_name = label_name.split('.')
        # Start with 0
        label_num = int(label_num) - 1
        label_dict[int(label_num)] = dog_name
        for image_name in os.listdir(path + '/' + label_name):
            img_paths.append(path + '/' + label_name + '/' + image_name)
            label_list.append(label_num)
    df = pd.DataFrame({'img_path': img_paths, 'label': label_list})
    return label_dict, df

In [7]:
num_label_dict, df_train = create_path_labels('dogImages/train')
_, df_val = create_path_labels('dogImages/valid')
_, df_test = create_path_labels('dogImages/test')

In [8]:
num_label_dict

{0: 'Affenpinscher',
 1: 'Afghan_hound',
 2: 'Airedale_terrier',
 3: 'Akita',
 4: 'Alaskan_malamute',
 5: 'American_eskimo_dog',
 6: 'American_foxhound',
 7: 'American_staffordshire_terrier',
 8: 'American_water_spaniel',
 9: 'Anatolian_shepherd_dog',
 10: 'Australian_cattle_dog',
 11: 'Australian_shepherd',
 12: 'Australian_terrier',
 13: 'Basenji',
 14: 'Basset_hound',
 15: 'Beagle',
 16: 'Bearded_collie',
 17: 'Beauceron',
 18: 'Bedlington_terrier',
 19: 'Belgian_malinois',
 20: 'Belgian_sheepdog',
 21: 'Belgian_tervuren',
 22: 'Bernese_mountain_dog',
 23: 'Bichon_frise',
 24: 'Black_and_tan_coonhound',
 25: 'Black_russian_terrier',
 26: 'Bloodhound',
 27: 'Bluetick_coonhound',
 28: 'Border_collie',
 29: 'Border_terrier',
 30: 'Borzoi',
 31: 'Boston_terrier',
 32: 'Bouvier_des_flandres',
 33: 'Boxer',
 34: 'Boykin_spaniel',
 35: 'Briard',
 36: 'Brittany',
 37: 'Brussels_griffon',
 38: 'Bull_terrier',
 39: 'Bulldog',
 40: 'Bullmastiff',
 41: 'Cairn_terrier',
 42: 'Canaan_dog',
 43: '

In [9]:
display(df_train.head())
print(df_train.shape)
print(df_val.shape)
print(df_test.shape)

,img_path,label
0,dogImages/train/001.Affenpinscher/Affenpinsche...,0
1,dogImages/train/001.Affenpinscher/Affenpinsche...,0
2,dogImages/train/001.Affenpinscher/Affenpinsche...,0
3,dogImages/train/001.Affenpinscher/Affenpinsche...,0
4,dogImages/train/001.Affenpinscher/Affenpinsche...,0


(6680, 2)
(835, 2)
(836, 2)


In [10]:
train_datagen = MultiOutputDataGenerator(images_paths=df_train['img_path'].values, labels=df_train['label'].values,
                              batch_size=batch_size, image_dimensions=image_shape, shuffle=True,
                              augmenter=create_augmenter(train=True), preprocessor=preprocess_input,
                             return_label=True, total_classes=total_classes, output_names=['original_out', 'se_out'])

val_datagen = MultiOutputDataGenerator(images_paths=df_val['img_path'].values, labels=df_val['label'].values,
                              batch_size=5, image_dimensions=image_shape, shuffle=True,
                              augmenter=None,
                                preprocessor=preprocess_input,
                                return_label=True, total_classes=total_classes, output_names=['original_out', 'se_out'])

test_datagen = MultiOutputDataGenerator(images_paths=df_test['img_path'].values, labels=df_test['label'].values,
                              batch_size=1, image_dimensions=image_shape, shuffle=False,
                              augmenter=None,
                                preprocessor=preprocess_input,
                            return_label=False, total_classes=total_classes, output_names=['original_out', 'se_out'])

In [11]:
print(len(train_datagen))
print(len(val_datagen))
print(len(test_datagen))

334
167
836


In [12]:
def squeeze_excite_block(tensor, ratio=16):
    # From: https://github.com/titu1994/keras-squeeze-excite-network
    init = tensor
    channel_axis = 1 if K.image_data_format() == "channels_first" else -1
    filters = K.int_shape(init)[channel_axis]
    se_shape = (1, 1, filters)

    se = GlobalAveragePooling2D()(init)
    se = Reshape(se_shape)(se)
    se = Dense(filters // ratio, activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
    se = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se)

    if K.image_data_format() == 'channels_first':
        se = Permute((3, 1, 2))(se)

    x = multiply([init, se])
    return x

In [13]:
# pooling = None - output shape (None, 8, 8, 1536)
# pooling = max  - output shape (None, 1536)
# pooling = avg  - output shape (None, 1536)

pretrained = MobileNetV2(include_top=False, weights='imagenet', input_shape=image_shape, pooling=None)
x = pretrained.output

# Original branch
gavg = GlobalAveragePooling2D()(x)
gmax = GlobalMaxPooling2D()(x)
original_concat = Concatenate(axis=-1)([gavg, gmax,])
original_concat = Dropout(0.5)(original_concat)
original_final = Dense(total_classes, activation='softmax', name='original_out')(original_concat)

# SE branch
se_out = squeeze_excite_block(x)
se_gavg = GlobalAveragePooling2D()(se_out)
se_gmax = GlobalMaxPooling2D()(se_out)
se_concat = Concatenate(axis=-1)([se_gavg, se_gmax,])
se_concat = Dropout(0.5)(se_concat)
se_final = Dense(total_classes, activation='softmax', name='se_out')(se_concat)

model = Model(inputs=pretrained.input, outputs=[original_final, se_final])

In [14]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
______________________________________________________________________________________________

In [15]:
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.MobileNetV2_warmup.hdf5', 
                                  verbose=1, save_best_only=True)


logdir = f".\logs\warmup"
# Create target Directory if don't exist
if not os.path.exists(logdir):
    os.mkdir(logdir)
    

tensorboard_callback = TensorBoard(log_dir=logdir)

early_stop = EarlyStopping(monitor="val_loss",
                               mode="min",
                               patience=15,
                               restore_best_weights=True)

# Maybe useful, haven't tried
def scheduler(epoch):
    if epoch < 50:
        return 0.001
    else:
        return 0.001 * tf.math.exp(0.1 * (10 - epoch))

lr_scheduler = LearningRateScheduler(scheduler)

## Warm up

In [16]:
# Freeze pretrained part
for layer in model.layers[:-15]:
    layer.trainable = False

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.MobileNetV2_best.hdf5', 
                                  verbose=1, save_best_only=True)

model.compile(optimizer='adam',
              loss={'original_out': 'categorical_crossentropy', 'se_out': 'categorical_crossentropy'},
              loss_weights={'original_out': 1., 'se_out': 1.}, metrics=['accuracy'])

In [17]:
history = model.fit_generator(generator=train_datagen,
                                       validation_data=val_datagen,
                                       epochs=10,
                                       callbacks=[tensorboard_callback, early_stop, checkpointer],
                                       verbose=1,
                                       )

Epoch 1/10
333/334 [============================>.] - ETA: 0s - loss: 17.8618 - original_out_loss: 12.8082 - se_out_loss: 5.0536 - original_out_accuracy: 0.0697 - se_out_accuracy: 0.1327 
Epoch 00001: val_loss improved from inf to 11.71253, saving model to saved_models/weights.best.MobileNetV2_warmup.hdf5
334/334 [==============================] - 117s 350ms/step - loss: 17.8541 - original_out_loss: 12.8046 - se_out_loss: 5.0494 - original_out_accuracy: 0.0701 - se_out_accuracy: 0.1331 - val_loss: 11.7125 - val_original_out_loss: 9.9487 - val_se_out_loss: 1.7639 - val_original_out_accuracy: 0.2491 - val_se_out_accuracy: 0.4958
Epoch 2/10
333/334 [============================>.] - ETA: 0s - loss: 14.1318 - original_out_loss: 11.3698 - se_out_loss: 2.7620 - original_out_accuracy: 0.1521 - se_out_accuracy: 0.3123  ETA: 56s - loss: 14.4802 - original_out_loss: 11.5544 
Epoch 00002: val_loss improved from 11.71253 to 10.24856, saving model to saved_models/weights.best.MobileNetV2_warmup.hdf

## Train whole model

In [18]:
# Freeze pretrained part
for layer in model.layers[:-15]:
    layer.trainable = True

logdir = f".\logs\whole"
# Create target Directory if don't exist
if not os.path.exists(logdir):
    os.mkdir(logdir)

tensorboard_callback = TensorBoard(log_dir=logdir)

sgd = SGD(lr=0.0005, momentum=0.0, decay=0.0, nesterov=True)

model.compile(optimizer=sgd,
              loss={'original_out': 'categorical_crossentropy', 'se_out': 'categorical_crossentropy'},
              loss_weights={'original_out': 1., 'se_out': 1.}, metrics=['accuracy'])

In [19]:
history = model.fit_generator(generator=train_datagen,
                                       validation_data=val_datagen,
                                       epochs=100,
                                       callbacks=[tensorboard_callback, early_stop, checkpointer],
                                       verbose=1,
                                       )

Epoch 1/100
333/334 [============================>.] - ETA: 0s - loss: 10.4090 - original_out_loss: 8.9538 - se_out_loss: 1.4552 - original_out_accuracy: 0.3541 - se_out_accuracy: 0.5913
Epoch 00001: val_loss improved from 9.07621 to 8.76601, saving model to saved_models/weights.best.MobileNetV2_warmup.hdf5
334/334 [==============================] - 125s 373ms/step - loss: 10.4062 - original_out_loss: 8.9502 - se_out_loss: 1.4559 - original_out_accuracy: 0.3545 - se_out_accuracy: 0.5913 - val_loss: 8.7660 - val_original_out_loss: 7.7782 - val_se_out_loss: 0.9878 - val_original_out_accuracy: 0.4455 - val_se_out_accuracy: 0.7066
Epoch 2/100
333/334 [============================>.] - ETA: 0s - loss: 9.7059 - original_out_loss: 8.4100 - se_out_loss: 1.2959 - original_out_accuracy: 0.3877 - se_out_accuracy: 0.6230
Epoch 00002: val_loss improved from 8.76601 to 8.62456, saving model to saved_models/weights.best.MobileNetV2_warmup.hdf5
334/334 [==============================] - 124s 371ms/ste

Epoch 28/100
333/334 [============================>.] - ETA: 0s - loss: 7.3018 - original_out_loss: 6.7237 - se_out_loss: 0.5781 - original_out_accuracy: 0.5357 - se_out_accuracy: 0.8183
Epoch 00028: val_loss did not improve from 7.46280
334/334 [==============================] - 112s 335ms/step - loss: 7.3025 - original_out_loss: 6.7253 - se_out_loss: 0.5772 - original_out_accuracy: 0.5358 - se_out_accuracy: 0.8187 - val_loss: 7.4990 - val_original_out_loss: 6.8950 - val_se_out_loss: 0.6040 - val_original_out_accuracy: 0.5234 - val_se_out_accuracy: 0.8084
Epoch 29/100
333/334 [============================>.] - ETA: 0s - loss: 7.3262 - original_out_loss: 6.7400 - se_out_loss: 0.5862 - original_out_accuracy: 0.5324 - se_out_accuracy: 0.8134
Epoch 00029: val_loss did not improve from 7.46280
334/334 [==============================] - 110s 329ms/step - loss: 7.3289 - original_out_loss: 6.7428 - se_out_loss: 0.5861 - original_out_accuracy: 0.5322 - se_out_accuracy: 0.8135 - val_loss: 7.488

334/334 [==============================] - 116s 346ms/step - loss: 3.2783 - original_out_loss: 2.2122 - se_out_loss: 1.0661 - original_out_accuracy: 0.5560 - se_out_accuracy: 0.7404 - val_loss: 3.1403 - val_original_out_loss: 2.4248 - val_se_out_loss: 0.7155 - val_original_out_accuracy: 0.5832 - val_se_out_accuracy: 0.7844
Epoch 42/100
333/334 [============================>.] - ETA: 0s - loss: 2.7648 - original_out_loss: 1.8250 - se_out_loss: 0.9398 - original_out_accuracy: 0.5874 - se_out_accuracy: 0.7529
Epoch 00042: val_loss improved from 3.14027 to 2.33389, saving model to saved_models/weights.best.MobileNetV2_warmup.hdf5
334/334 [==============================] - 121s 364ms/step - loss: 2.7617 - original_out_loss: 1.8228 - se_out_loss: 0.9388 - original_out_accuracy: 0.5877 - se_out_accuracy: 0.7533 - val_loss: 2.3339 - val_original_out_loss: 1.6228 - val_se_out_loss: 0.7111 - val_original_out_accuracy: 0.6479 - val_se_out_accuracy: 0.7832
Epoch 43/100
333/334 [===================

Epoch 55/100
333/334 [============================>.] - ETA: 0s - loss: 1.2052 - original_out_loss: 0.7194 - se_out_loss: 0.4858 - original_out_accuracy: 0.8071 - se_out_accuracy: 0.8569
Epoch 00055: val_loss did not improve from 1.25448
334/334 [==============================] - 128s 383ms/step - loss: 1.2077 - original_out_loss: 0.7205 - se_out_loss: 0.4872 - original_out_accuracy: 0.8066 - se_out_accuracy: 0.8566 - val_loss: 1.3013 - val_original_out_loss: 0.7471 - val_se_out_loss: 0.5542 - val_original_out_accuracy: 0.8120 - val_se_out_accuracy: 0.8240
Epoch 56/100
333/334 [============================>.] - ETA: 0s - loss: 1.1774 - original_out_loss: 0.7018 - se_out_loss: 0.4757 - original_out_accuracy: 0.8081 - se_out_accuracy: 0.8610
Epoch 00056: val_loss did not improve from 1.25448
334/334 [==============================] - 129s 387ms/step - loss: 1.1792 - original_out_loss: 0.7024 - se_out_loss: 0.4769 - original_out_accuracy: 0.8081 - se_out_accuracy: 0.8606 - val_loss: 1.310

Epoch 70/100
333/334 [============================>.] - ETA: 0s - loss: 0.8333 - original_out_loss: 0.4731 - se_out_loss: 0.3602 - original_out_accuracy: 0.8662 - se_out_accuracy: 0.8878
Epoch 00070: val_loss did not improve from 1.22951
334/334 [==============================] - 116s 348ms/step - loss: 0.8319 - original_out_loss: 0.4724 - se_out_loss: 0.3595 - original_out_accuracy: 0.8665 - se_out_accuracy: 0.8882 - val_loss: 1.2843 - val_original_out_loss: 0.7570 - val_se_out_loss: 0.5273 - val_original_out_accuracy: 0.8299 - val_se_out_accuracy: 0.8311
Epoch 71/100
333/334 [============================>.] - ETA: 0s - loss: 0.8188 - original_out_loss: 0.4789 - se_out_loss: 0.3399 - original_out_accuracy: 0.8649 - se_out_accuracy: 0.8986
Epoch 00071: val_loss did not improve from 1.22951
334/334 [==============================] - 119s 355ms/step - loss: 0.8174 - original_out_loss: 0.4780 - se_out_loss: 0.3394 - original_out_accuracy: 0.8650 - se_out_accuracy: 0.8990 - val_loss: 1.282

In [20]:
model.save_weights('saved_models/weights.best.MobileNetV2_whole_model.hdf5')

In [31]:
K.clear_session()

In [22]:
model.load_weights('saved_models/weights.best.MobileNetV2_whole_model.hdf5')

In [23]:
pred = model.predict_generator(generator=test_datagen,
                                       verbose=1,
                                       )

836/836 [==============================] - 45s 53ms/step


In [19]:
def combine_prediction(predictions, weights=[1.,1.]):
    predictions = np.array(predictions)
    weights = np.array(weights).reshape(predictions.shape[0], 1, 1)
    return np.mean(predictions * weights, axis=0)

def cal_accuracy(predictions, truth):
    if type(predictions) != list:
        predictions = [predictions]
    accuracy = []
    for prediction in predictions:
        prediction = np.argmax(prediction, axis=-1)
        correct_nums = (prediction == truth).sum()
        accuracy.append(correct_nums / len(prediction))
    return accuracy

In [25]:
# Same weight
cal_accuracy(combine_prediction(pred, [1., 1.]), df_test['label'].values)

[0.8325358851674641]

## Load best model

In [32]:
model.load_weights('saved_models/weights.best.MobileNetV2_best.hdf5')

In [33]:
pred = model.predict_generator(generator=test_datagen,
                                       verbose=1,
                                       )

836/836 [==============================] - 40s 47ms/step


In [34]:
len(df_test['label'].values)

836

In [35]:
# Same weight
cal_accuracy(combine_prediction(pred, [1., 1.]), df_test['label'].values)

[0.8325358851674641]

## Test Time Augmentation

In [36]:
# Get Augmentors
tta_augmentors = [iaa.Fliplr(1.), iaa.Flipud(1.)]


In [37]:
tta_test_datagen = MultiOutputDataGenerator(images_paths=df_test['img_path'].values, labels=df_test['label'].values,
                              batch_size=1, image_dimensions=image_shape, shuffle=False,
                              augmenter=None,
                                preprocessor=preprocess_input,
                            return_label=False, total_classes=total_classes, output_names=['original_out', 'se_out'], tta_augmentors=tta_augmentors)

In [38]:
total_classes

133

In [39]:
all_predictions = np.zeros((df_test.shape[0], total_classes))
count = 0
total_len = len(tta_test_datagen)
for images in tta_test_datagen:
    count += 1
    print("{}/{}".format(count, total_len), end="\r")
    preds = []
    for image in images:
        pred = model.predict_on_batch(image)
        pred = combine_prediction(pred, [1.0, 1.0])
        preds.append(pred)
    all_predictions[count-1] = combine_prediction(preds, [1.0, 1.0, 1.0])

In [40]:
cal_accuracy(np.array(all_predictions).reshape((836, 133)), df_test['label'].values)

[0.7464114832535885]

## Pseudo labeling

In [51]:
val_pred = model.predict_generator(generator=val_datagen,
                                       verbose=1,
                                       )


167/167 [==============================] - 9s 53ms/step


In [71]:
val_pred = combine_prediction(val_pred, [1., 1.])

In [72]:
test_datagen = MultiOutputDataGenerator(images_paths=df_test['img_path'].values, labels=df_test['label'].values,
                              batch_size=2, image_dimensions=image_shape, shuffle=False,
                              augmenter=None,
                                preprocessor=preprocess_input,
                            return_label=False, total_classes=total_classes, output_names=['original_out', 'se_out'])

test_pred = model.predict_generator(generator=test_datagen,
                                       verbose=1,
                                       )

test_pred = combine_prediction(test_pred, [1., 1.])

418/418 [==============================] - 22s 54ms/step


In [73]:
val_pred.shape

(835, 133)

In [74]:
test_pred.shape

(836, 133)

In [75]:
val_pseudolabels= np.argmax(val_pred, axis=-1)
test_pseudolabels = np.argmax(test_pred, axis=-1)

In [76]:
print(val_pseudolabels.shape)

print(test_pseudolabels.shape)

(835,)
(836,)


### Combine pseudo labels with train data

In [77]:
pseudo_data = pd.DataFrame({"img_path": np.concatenate([df_val['img_path'].values, df_test['img_path'].values]),
                            "label": np.concatenate([val_pseudolabels, test_pseudolabels])})

In [78]:
pseudo_train = pd.concat([pseudo_data, df_train], axis=0)

In [79]:
pseudo_train.shape

(8351, 2)

In [80]:
pseudo_train_datagen = MultiOutputDataGenerator(images_paths=pseudo_train['img_path'].values, labels=pseudo_train['label'].values,
                              batch_size=batch_size, image_dimensions=image_shape, shuffle=True,
                              augmenter=create_augmenter(train=True), preprocessor=preprocess_input,
                             return_label=True, total_classes=total_classes, output_names=['original_out', 'se_out'])

In [81]:
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.MobileNetV2_PL.hdf5', 
                                  verbose=1, save_best_only=True)

logdir = f".\logs\pseudolabeling"
# Create target Directory if don't exist
if not os.path.exists(logdir):
    os.mkdir(logdir)

tensorboard_callback = TensorBoard(log_dir=logdir)

early_stop = EarlyStopping(monitor="val_loss",
                               mode="min",
                               patience=15,
                               restore_best_weights=True)

In [82]:
model.load_weights('saved_models/weights.best.MobileNetV2_whole_model.hdf5')

In [84]:
# Freeze pretrained part
for layer in model.layers[:-15]:
    layer.trainable = True
    
sgd = SGD(lr=0.0001, momentum=0.0, decay=0.0, nesterov=True)

model.compile(optimizer=sgd,
              loss={'original_out': 'categorical_crossentropy', 'se_out': 'categorical_crossentropy'},
              loss_weights={'original_out': 1., 'se_out': 1.}, metrics=['accuracy'])

In [85]:
history = model.fit_generator(generator=pseudo_train_datagen,
                                       validation_data=val_datagen,
                                       epochs=100,
                                       callbacks=[tensorboard_callback, early_stop, checkpointer, lr_scheduler],
                                       verbose=1,
                                       )

Epoch 1/100
416/417 [============================>.] - ETA: 0s - loss: 3.4529 - original_out_loss: 1.8777 - se_out_loss: 1.5752 - original_out_accuracy: 0.7123 - se_out_accuracy: 0.7475
Epoch 00001: val_loss improved from inf to 1.32282, saving model to saved_models/weights.best.MobileNetV2_PL.hdf5
417/417 [==============================] - 143s 343ms/step - loss: 3.4560 - original_out_loss: 1.8793 - se_out_loss: 1.5767 - original_out_accuracy: 0.7124 - se_out_accuracy: 0.7474 - val_loss: 1.3228 - val_original_out_loss: 0.7056 - val_se_out_loss: 0.6172 - val_original_out_accuracy: 0.7976 - val_se_out_accuracy: 0.8120
Epoch 2/100
416/417 [============================>.] - ETA: 0s - loss: 3.3045 - original_out_loss: 1.7560 - se_out_loss: 1.5485 - original_out_accuracy: 0.7149 - se_out_accuracy: 0.7466
Epoch 00002: val_loss did not improve from 1.32282
417/417 [==============================] - 144s 345ms/step - loss: 3.3046 - original_out_loss: 1.7563 - se_out_loss: 1.5483 - original_out

Epoch 16/100
416/417 [============================>.] - ETA: 0s - loss: 2.2207 - original_out_loss: 1.1193 - se_out_loss: 1.1014 - original_out_accuracy: 0.7831 - se_out_accuracy: 0.8004
Epoch 00016: val_loss did not improve from 1.32282
417/417 [==============================] - 143s 344ms/step - loss: 2.2239 - original_out_loss: 1.1202 - se_out_loss: 1.1036 - original_out_accuracy: 0.7826 - se_out_accuracy: 0.8000 - val_loss: 2.7713 - val_original_out_loss: 1.4434 - val_se_out_loss: 1.3280 - val_original_out_accuracy: 0.6359 - val_se_out_accuracy: 0.7042


In [86]:
pred = model.predict_generator(generator=test_datagen,
                                       verbose=1,
                                       )

418/418 [==============================] - 21s 49ms/step


In [87]:
# Same weight
cal_accuracy(combine_prediction(pred, [1., 1.]), df_test['label'].values)

[0.8301435406698564]

## Load best model

In [88]:
model.load_weights('saved_models/weights.best.MobileNetV2_PL.hdf5')

In [89]:
pred = model.predict_generator(generator=test_datagen,
                                       verbose=1,
                                       )
# Same weight
cal_accuracy(combine_prediction(pred, [1., 1.]), df_test['label'].values)

418/418 [==============================] - 21s 51ms/step


[0.8301435406698564]

# Modify model for deploy

In [16]:
model.load_weights('saved_models/weights.best.MobileNetV2_whole_model.hdf5')
sgd = SGD(lr=0.0001, momentum=0.0, decay=0.0, nesterov=True)

model.compile(optimizer=sgd,
              loss={'original_out': 'categorical_crossentropy', 'se_out': 'categorical_crossentropy'},
              loss_weights={'original_out': 1., 'se_out': 1.}, metrics=['accuracy'])

In [26]:
combined_output = Average()(model.output)
combined_model = Model(inputs=model.input, outputs=combined_output)

In [27]:
combined_model.output

<tf.Tensor 'average/Identity:0' shape=(None, 133) dtype=float32>

In [28]:
test_datagen = BaseDataGenerator(images_paths=df_test['img_path'].values, labels=df_test['label'].values,
                              batch_size=1, image_dimensions=image_shape, shuffle=False,
                              augmenter=None,
                                preprocessor=preprocess_input,
                            return_label=False, total_classes=total_classes)

In [31]:
pred = combined_model.predict_generator(generator=test_datagen,
                                       verbose=1,
                                       )

836/836 [==============================] - 13s 15ms/step


In [32]:
cal_accuracy(pred, df_test['label'].values)

[0.8325358851674641]

In [33]:
combined_model.save_weights('saved_models/weights.best.MobileNetV2_combined.hdf5')